In [ ]:
#@title Rode essa cédula para montar o seu Drive.
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

In [ ]:
#@title Rode essa cédula para baixar o ESRGAN diretamente em seu Google Drive. (Se você ainda não tiver)

%cd "/content/drive/My Drive"
!git clone https://github.com/styler00dollar/Colab-ESRGAN.git
!mkdir "/content/drive/My Drive/Colab-ESRGAN/input/"
!mkdir "/content/drive/My Drive/Colab-ESRGAN/output/"
!mkdir "/content/drive/My Drive/Colab-ESRGAN/models/"
%cd "/content/drive/My Drive/Colab-ESRGAN"

### A seguir você poderá baixar alguns modelos de Upscale diretamente para a pasta "/content/drive/My Drive/Colab-ESRGAN/models" em seu Google Drive. Existem vários outros modelos que você pode encontrar pela internet. Se preferir, coloque o modelo manualmente a partir do seu Google Drive. 

In [ ]:
#@title Download ESRGANx4 and PSNRx4
%cd "/content/drive/My Drive/Colab-ESRGAN"
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-' -O models/RRDB_ESRGAN_x4.pth
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr' -O models/RRDB_PSNR_x4.pth

In [ ]:
#@title Downlaod fatal_anime (Model by twittman)
%cd "/content/drive/My Drive/Colab-ESRGAN"
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download" -O models/4x_fatal_Anime_500000_G.pth

In [ ]:
#@title Downlaod fatal_pixels (Model by twittman)
%cd "/content/drive/My Drive/Colab-ESRGAN"
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/mDGmi7NgdyyQRXL/download?path=%2F&files=4x_FatalPixels_340000_G.pth&downloadStartSecret=r4q3aw60ijm" -O models/4x_FatalPixels_340000_G.pth

In [ ]:
#@title Download deJpeg_Fatality_PlusULTRA (Model by twittman)
%cd "/content/drive/My Drive/Colab-ESRGAN"
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download" -O models/1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth


Modelos disponíveis (se você fez o download de todos):
- RRDB_ESRGAN_x4.pth (old-arch)
- RRDB_PSNR_x4.pth (old-arch)
- 4x_FatalPixels_340000_G.pth
- 4x_fatal_Anime_500000_G.pth
- 1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth

In [ ]:
#@title Seleciona o vídeo e captura taxa de FPS
import os


caminho = "/content/drive/My Drive/"
diretorio = os.listdir(caminho)
indice = 1

for item in diretorio:
  print([indice], item)
  indice += 1
    
print()
novoindice = int(input("\nSelecione o seu arquivo: ")) - 1
fps = !ffmpeg -i "{caminho}{diretorio[novoindice]}"
fps = [x for x in str(fps).split(",") if "fps" in x][0].split()[0]
!rm "fps.txt" 2>/dev/null

In [ ]:
#@title Cédula para extrair o áudio do vídeo
print("Extraindo audio ...")
!ffmpeg -i "{caminho}{diretorio[novoindice]}" -vn -acodec copy /content/audio.aac 2>/dev/null
size = !du -sh /content/audio.aac  2>/dev/null
if size[0].split()[0] == '0':
  !rm /content/audio.aac
  !ffmpeg -i  "{caminho}{diretorio[novoindice]}" -vn -ac 2 -ar 44100 -ab 320k -f mp3 /content/audio.aac  2>/dev/null
print('Concluído...')

In [ ]:
#@title Extrai frames do vídeo
print("Extraindo frames do seu vídeo...")

%cd /content/drive/My Drive/Colab-ESRGAN/input 
!ffmpeg -i "{caminho}{diretorio[novoindice]}" %d.png 2>/dev/null

In [ ]:
#@title Instala a dependência "tqdm" e entra na pasta do ESRGAN em seu Drive
!pip install tqdm
%cd "/content/drive/My Drive/Colab-ESRGAN"

In [ ]:
# COMBINANDO MODELOS
!python main.py input output --device cuda --prefilter models/1x_Fatality_DeBlur_270000_G.pth --model models/4x_DigitalFrames_2.1_Aggressive.pth 

In [ ]:
# EXEMPLO DE USO COM APENAS UM MODELO; Manga109Attempt
!python main.py input/ output/ --device cuda --model models/Manga109Attempt.pth

In [ ]:
# EXEMPLO DE USO COM APENAS UM MODELO; 1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth
!python main.py input/ output/ --device cuda --model models/1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth

In [ ]:
# INTERPOLANDO
!python main.py input output --device cuda --model models/RRDB_PSNR_x4.pth:70:models/RRDB_ESRGAN_x4.pth:70

In [ ]:
# example for model interpolation
# recommended tilesize for 16GB and an interpolated 4x model: --tilesize 2000
!python main.py input/ output/ --device cuda --model models/4x_FatalPixels_340000_G.pth:25:models/4x_fatal_Anime_500000_G.pth:75

**ESSA CÉDULA SERVE PARA GERAR O VÍDEO. VOCÊ DEVE POR O NOME DO MODELO ANTES. POR PADRÃO EU DEIXEI O 1x_DeJpeg_Fatality_PlusULTRA_200000_G.**



In [ ]:
!ffmpeg -f image2 -r  {fps} -i output/1x_DeJpeg_Fatality_PlusULTRA_200000_G/%d.png -i /content/audio.aac "/content/drive/My Drive/{diretorio[novoindice][0:-4]}_upscale.mkv" 

### Essa ultima cédulas você poderá usar caso queira que todas as imagens em ```/output``` reduzam a 50% da sua resolução




In [ ]:
#@title Reduzir imagens do /output a 50%
!apt-get install imagemagick
!find "/content/drive/My Drive/Colab-ESRGAN/output" -name '*.png' -exec mogrify -resize 50% {} +